In [1]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.2
    Uninstalling httpx-sse-0.4.2:
      Successfully uninstalled httpx-sse-0.4.2


In [2]:
import cohere
co = cohere.ClientV2("WuOIVkz19Iv8kNTzDDce4dYxUHIWuiQPqTw7hCI1") # Get your free API key: https://dashboard.cohere.com/api-keys

In [3]:
import os
import json
import numpy as np
import pandas as pd
from cohere import ClassifyExample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
# Load the dataset to a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/atis_subset.csv', names=['query','intent'])

In [5]:
# Split the dataset into training and test portions
df_train, df_test = train_test_split(df, test_size=200, random_state=21)

In [6]:
['atis_flight',
 'atis_airfare',
 'atis_ground_service',
 'atis_flight_time',
 'atis_airline',
 'atis_quantity',
 'atis_abbreviation',
 'atis_aircraft']

['atis_flight',
 'atis_airfare',
 'atis_ground_service',
 'atis_flight_time',
 'atis_airline',
 'atis_quantity',
 'atis_abbreviation',
 'atis_aircraft']

In [7]:
def create_classification_data(text, label):
    formatted_data = {
        "text": text,
        "label": label
    }
    return formatted_data

if not os.path.isfile("data.jsonl"):
    print("Creating jsonl file ...")
    with open("data.jsonl", 'w+') as file:
        for row in df_train.itertuples():
            formatted_data = create_classification_data(row.query, row.intent)
            file.write(json.dumps(formatted_data) + '\n')
        file.close()
        print("Done")
else:
    print("data.jsonl file already exists")

Creating jsonl file ...
Done


In [5]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 60.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Tải dữ liệu
df = pd.read_csv(
    'https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/atis_subset.csv',
    names=['query', 'intent']
)

# 2. Tách tập train / test
df_train, df_test = train_test_split(df, test_size=0.2, random_state=21)


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Chuẩn bị dữ liệu
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['query'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("intent", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("intent", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Label mapping
labels = df['intent'].unique()
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

def encode_labels(example):
    example['labels'] = label2id[example['labels']]
    return example

train_dataset = train_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

# Load model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

# Huấn luyện
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tinhocplus16 (tinhocplus16-university-of-transport-comunication) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [cohere] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Epoch,Training Loss,Validation Loss
1,No log,0.450780
2,No log,0.273004
3,No log,0.194184


TrainOutput(global_step=300, training_loss=0.36168065388997395, metrics={'train_runtime': 275.9584, 'train_samples_per_second': 8.697, 'train_steps_per_second': 1.087, 'total_flos': 631500550963200.0, 'train_loss': 0.36168065388997395, 'epoch': 3.0})

In [13]:
!ls /content/results

checkpoint-300	runs


In [15]:
model_path = "/content/results/checkpoint-300"
print(model_path)

/content/results/checkpoint-300


In [16]:
!ls /content/results/checkpoint-300

config.json	   optimizer.pt   scheduler.pt	      training_args.bin
model.safetensors  rng_state.pth  trainer_state.json


In [17]:
from transformers import BertTokenizer

# ✅ Load tokenizer gốc
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 💾 Lưu tokenizer vào thư mục checkpoint của bạn
tokenizer.save_pretrained("/content/results/checkpoint-300")

('/content/results/checkpoint-300/tokenizer_config.json',
 '/content/results/checkpoint-300/special_tokens_map.json',
 '/content/results/checkpoint-300/vocab.txt',
 '/content/results/checkpoint-300/added_tokens.json')

In [18]:
!ls /content/results/checkpoint-300

config.json	   rng_state.pth	    tokenizer_config.json  vocab.txt
model.safetensors  scheduler.pt		    trainer_state.json
optimizer.pt	   special_tokens_map.json  training_args.bin


In [19]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "/content/results/checkpoint-300"

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [20]:
import torch

# Một câu ví dụ (bạn có thể đổi tuỳ ý)
text = "show me flights from boston to dallas"

# Tokenize đầu vào
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Chạy mô hình để lấy kết quả dự đoán
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    pred_id = torch.argmax(logits, dim=1).item()

# In ra ID class và tên nhãn
print("Predicted class ID:", pred_id)
print("Predicted label:", model.config.id2label[pred_id])

Predicted class ID: 0
Predicted label: atis_flight


In [21]:
from sklearn.metrics import accuracy_score, classification_report

texts = df_test['query'].tolist()
labels = df_test['intent'].tolist()

# map từ label sang id
label2id = {v: k for k, v in model.config.id2label.items()} if hasattr(model.config, "id2label") else None
if label2id is None:
    # nếu chưa có mapping thì tự tạo từ tập train
    unique_labels = sorted(df_test['intent'].unique())
    label2id = {label: i for i, label in enumerate(unique_labels)}

y_true = [label2id[label] for label in labels]
y_pred = []

for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=1).item()
    y_pred.append(pred_id)

# Tính độ chính xác
acc = accuracy_score(y_true, y_pred)
print("✅ Test Accuracy:", acc)

# Báo cáo chi tiết theo từng class
print(classification_report(y_true, y_pred, target_names=label2id.keys()))

✅ Test Accuracy: 0.965
                     precision    recall  f1-score   support

        atis_flight       0.99      0.99      0.99       147
   atis_flight_time       0.50      1.00      0.67         1
       atis_airfare       1.00      0.92      0.96        12
      atis_aircraft       0.86      1.00      0.92         6
atis_ground_service       0.86      1.00      0.92        12
       atis_airline       1.00      0.67      0.80         6
  atis_abbreviation       0.85      0.85      0.85        13
      atis_quantity       1.00      1.00      1.00         3

           accuracy                           0.96       200
          macro avg       0.88      0.93      0.89       200
       weighted avg       0.97      0.96      0.97       200

